In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
!pip install wget
import wget
!pip install mne
!pip install pymatreader
import mne
import numpy as np
import pickle
from scipy.signal import welch

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=3bcaaa48887be6be8d6b701a236dfed2bf35496d9846186ad624ed8b2d421d12
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 8.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Remoção do artefato ocular

In [ ]:
def remover_artefato(EEG, olho):

    sample_no = len(EEG)
    order = 3
    sigma = 0.01
    lambda_ = 0.9999
    H = np.zeros((order, 1))
    R = sigma * np.eye(order, order)

    EEGlimpo = np.zeros(sample_no)
    for n in range(sample_no):
        s = EEG[n]
        if n >= order-1:
            j = 1          # calculation of last "order" reference vector
            r = np.zeros((order, 1))
            for i in range(n, n-order, -1):
                r[j-1, 0] = olho[i]
                j = j+1
            # Calculation of Filter Coefficients
            K = np.linalg.inv(R) @ r @ np.linalg.inv(lambda_ + r.T @ np.linalg.inv(R) @ r)
            e = s - (r.T @ H)
            H = H + (K * e)
            R = np.linalg.inv((1/lambda_ * np.linalg.inv(R)) - ((1/lambda_) * K * (r.T) @ np.linalg.inv(R)))
            # Calculation Of RLS algorithm estimated signal
            EEGlimpo[n] = s - (r.T @ H)


    return EEGlimpo

In [ ]:
for i in range(4,11):
  istring = str(i).zfill(3)
  canais = ['Fp1', 'Fpz', 'Fp2', 'AFp1', 'AFp2', 'AF3', 'AFz', 'AF4', 'AFF1h',
            'AFF2h', 'F1', 'F2', 'FFC1h', 'FFC2h', 'FC1', 'FCz', 'FC2', 'FCC1h',
            'FCC2h', 'C1', 'Cz', 'C2', 'CCP1h', 'CCP2h', 'CP1', 'CPz', 'CP2',
            'CPP1h', 'CPP2h', 'P1', 'Pz', 'P2', 'PPO1h', 'PPO2h', 'PO3', 'POz',
            'PO4', 'O1', 'Oz', 'O2', 'I1', 'Iz', 'I2', 'AF7', 'F9', 'F7', 'AF7',
            'F9', 'F7', 'FT9', 'AFF5h', 'F5', 'FT7', 'F3', 'FFC5h', 'FC5',
            'FTT7h', 'T7', 'FFC3h', 'FC3', 'FCC5h', 'C5', 'TTP7h', 'TP7', 'P9',
            'FCC3h', 'C3', 'CCP5h', 'CP5', 'P7', 'PO9', 'CCP3h', 'CP3', 'CPP5h',
            'P5', 'PO7', 'CPP3h', 'P3', 'AF8', 'F10', 'F8', 'FT10', 'AFF6h',
            'F6', 'FT8', 'F4', 'FFC6h', 'FC6', 'FTT8h', 'T8', 'FFC4h', 'FC4',
            'FCC6h', 'C6', 'TTP8h', 'TP8', 'P10', 'FCC4h', 'C4', 'CCP6h', 'CP6',
            'P8', 'PO10', 'CCP4h', 'CP4', 'CPP6h', 'P6', 'PO8', 'CPP4h', 'P4']

  eeg_limpo = dict()
  Peeg_limpo = dict()

  for run in range(1, 11):
      runstring = str(run).zfill(2)

      wget.download('https://openneuro.org/crn/datasets/ds001971/snapshots/1.1.1/files/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.fdt')
      wget.download('https://openneuro.org/crn/datasets/ds001971/snapshots/1.1.1/files/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
      data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
      olho = (-data.get_data('Fp1').T - data.get_data('Fp2').T)/2

      for j in range(len(canais)):
          print(j)
          eeg = data.get_data(canais[j]).T
          eeg_limpo[canais[j]] = remover_artefato(eeg, olho)
          f,Peeg_limpo[canais[j]] = welch(eeg_limpo[canais[j]], fs=512)
      with open('/content/drive/MyDrive/PDPDFernanda2022/arquivosEEG/subj'+str(i)+'run'+runstring+'.pkl', 'wb') as handle:
          pickle.dump(eeg_limpo, handle, protocol=pickle.HIGHEST_PROTOCOL)
      with open('/content/drive/MyDrive/PDPDFernanda2022/arquivosEEG_espectro/subj'+str(i)+'run'+runstring+'.pkl', 'wb') as handle:
          pickle.dump(Peeg_limpo, handle, protocol=pickle.HIGHEST_PROTOCOL)




Reading /content/sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-01_eeg.fdt


<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Estimated head radius (0.1 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Data file name in EEG.data (sub-004_task-AudioCueWalkingStudy_run-01_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-01_eeg.fdt).
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Not setting positions of 4 emg channels found in montage:
['TIBR1', 'TIBR2', 'TIBL1', 'TIBL2']
Consider setting the channel types to be of EEG/sEEG/ECoG/

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
Reading /content/sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-02_eeg.fdt


<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Unknown types found, setting as type EEG:
aux: ['ANKLE', 'HIP', 'KNEE']
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Data file name in EEG.data (sub-004_task-AudioCueWalkingStudy_run-02_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-02_eeg.fdt).
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Not setting positions of 4 emg channels found in montage:
['TIBR1', 'TIBR2', 'TIBL1', 'TIBL2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  data = m

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
Reading /content/sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-03_eeg.fdt


...

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
Reading /content/sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-04_eeg.fdt


<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Unknown types found, setting as type EEG:
aux: ['ANKLE', 'HIP', 'KNEE']
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Data file name in EEG.data (sub-004_task-AudioCueWalkingStudy_run-04_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-04_eeg.fdt).
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Not setting positions of 4 emg channels found in montage:
['TIBR1', 'TIBR2', 'TIBL1', 'TIBL2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  data = m

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
Reading /content/sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-05_eeg.fdt


<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Unknown types found, setting as type EEG:
aux: ['ANKLE', 'HIP', 'KNEE']
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Data file name in EEG.data (sub-004_task-AudioCueWalkingStudy_run-05_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-004:eeg:sub-004_task-AudioCueWalkingStudy_run-05_eeg.fdt).
  data = mne.io.read_raw_eeglab('/content/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.set')
<ipython-input-4-9bbedb0e03bf>:24: RuntimeWarning: Not setting positions of 4 emg channels found in montage:
['TIBR1', 'TIBR2', 'TIBL1', 'TIBL2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  data = m

0
1
2
3
4
5
6
7
8


In [ ]:
'https://openneuro.org/crn/datasets/ds001971/snapshots/1.1.1/files/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.fdt'

In [ ]:
'https://openneuro.org/crn/datasets/ds001971/snapshots/1.1.1/files/sub-'+istring+':eeg:sub-'+istring+'_task-AudioCueWalkingStudy_run-'+runstring+'_eeg.fdt'